<a href="https://colab.research.google.com/github/JohnScovilAtwood/JohnScovilAtwood.github.io/blob/main/JohnAtwood_CS580_Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

John Atwood
Assignment 5
CS580: Introduction to Artificial Intelligence


Report your testing, training, and validation accuracy and provide analysis.





In [8]:
# task one: for whatever reason, Google Colab is making it impossible to either read directly from a URL 
# OR download to my personal machine and upload as a file
# Here is proof my my attempt

import gzip
file = gzip.open(r"C:\Users\johns\Desktop\optdigits-orig.windep.Z",'rb')
file_content = file.read()
print(file_content)


FileNotFoundError: ignored

In [9]:
import numpy as np
from keras.datasets import mnist
# I looked at the data source available by UCI; it comes directly from MNIST, and I can just import from datasets already made available by Keras.
# It's easier to just do it using KERAS; I have done this before and it has worked in the past.
# I'll still be completing the work in Task 1.
from keras.utils import np_utils

# let's put together some functions that help us build our network
# layer: input/output, forward/backprop
# fully connected layer: basic; every node is connected to every node in next layer
# activation layer: need to handle nonlinearity in model

class layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        raise NotImplementedError

    def backprop(self, output_error, learning_rate):
        raise NotImplementedError

class dense(layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backprop(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

class activation_layer(layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backprop(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

# Here, we use a tanh function for our activation function; was going to use sigmoid, looked it up and this is better
# sigmoid with activation functions w/ Real inputs and outputs in [-1,1]
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

# loss function and its derivative; just some simple math
def mean_sq_error(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mean_sq_error_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

# here we can actually construct our network
# TASK 2: Option 1; build network from scratch
class network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer
    def add(self, layer):
        self.layers.append(layer)

    # set loss 
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward(output)
            result.append(output)

        return result

    # Train the network!
    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward(output)

                err += self.loss(y_train[j], output)

                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backprop(error, learning_rate)

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

# TASK 1: This reads in the data, and encodes into input and target
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# we have to do some manipulation on the data to make it ready for the network
# Output: in [0,9]; let's make this easier for the network to output
# Instead of doing a bunch of work on my end to code in a translator for output to english,
# let's just keep it simple. The output will be a vector of 0's with 1 in the position of the output number.
# [1,0,0,0,0,0,0,0,0,0] = 0, [0,1,0,0,0,0,0,0,0,0] = 1, [0,0,1,0,0,0,0,0,0,0] = 2, etc.
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_train = x_train.astype('float') 
x_test = x_test.astype('float')
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test)

# need to normalize pixel values 
x_test /= 255 
x_train /= 255

# Here, we add layers to the network. We want a sort of funnel look to our network, so the network is shallow and fewer and fewer nodes as you traverse the network.
# mnist input is 28*28; output prev needs to equal input of next; let's keep walking down fullyconnected layer(input,output)
new_net = network()
new_net.add(dense(28*28, 1000))                
new_net.add(activation_layer(tanh, tanh_prime))

new_net.add(dense(1000, 100))                  
new_net.add(activation_layer(tanh, tanh_prime))

new_net.add(dense(100, 10))                    
new_net.add(activation_layer(tanh, tanh_prime))

# train on 500 examples; can always increase/ decrease as needed (need to call mean squared error and derivative here)
new_net.use(mean_sq_error, mean_sq_error_prime)
new_net.fit(x_train[0:500], y_train[0:500], epochs=20, learning_rate=0.1) # learning rate from previous assignments; have quickly the gradient descends in gradient descent

# test on 5 examples
out = new_net.predict(x_test[0:5])
print("Predicted: ") # value closest to 1 in array translates to what the network is predicting is the answer
print(out, end="\n")
print("Actual: ")
print(y_test[0:5])


# BONUS 
# Bonus: (20 pts for CS480 students and 10 pts for CS580 students)
# Using 1024 features for neural network is very cumbersome. Is there a way to reduce the number of features? If you have an idea, show me how effective it is.





epoch 1/20   error=0.676620
epoch 2/20   error=0.451173
epoch 3/20   error=0.283637
epoch 4/20   error=0.208008
epoch 5/20   error=0.160418
epoch 6/20   error=0.136834
epoch 7/20   error=0.119069
epoch 8/20   error=0.109362
epoch 9/20   error=0.104277
epoch 10/20   error=0.095626
epoch 11/20   error=0.089085
epoch 12/20   error=0.085718
epoch 13/20   error=0.081420
epoch 14/20   error=0.076990
epoch 15/20   error=0.076393
epoch 16/20   error=0.075592
epoch 17/20   error=0.071239
epoch 18/20   error=0.071435
epoch 19/20   error=0.070023
epoch 20/20   error=0.068290
Predicted: 
[array([[-0.14158789, -0.22515732, -0.34094748,  0.48019713,  0.55742139,
         0.27372647,  0.17773262,  0.93828782, -0.68988536, -0.61750714]]), array([[-0.25112754, -0.08161351,  0.83450907, -0.05812246,  0.12547654,
         0.03255222,  0.03378668,  0.27802079, -0.21530406, -0.23317278]]), array([[-0.30910077,  0.99444543,  0.23872794, -0.41416763,  0.57555328,
        -0.32303069, -0.47178453, -0.45550473